In [2]:
import numpy as np
import h5py
import matplotlib
matplotlib.use("Agg")
import sys, os, numpy
import matplotlib.pyplot as plt
import tensorflow
from sklearn.model_selection import train_test_split

from qkeras.utils import load_qmodel

os.environ['PATH'] = '/data/software/xilinx/Vivado/2020.1/bin:' + os.environ['PATH'] #Mulder

model = load_qmodel("/home/users/russelld/L1JetTagDaniel/hls4mlModifications/10-08-23/qkL1JetTagModel.h5")

#Convert model to HLS

import hls4ml
config = hls4ml.utils.config_from_keras_model(model, granularity='name')
print("-----------------------------------")

config['LayerName']['q_conv1d']['ReuseFactor'] = 2
config['LayerName']['q_conv1d_1']['ReuseFactor'] = 2
config["LayerName"]["q_conv1d"]["ConvImplementation"] = "Pointwise"
config["LayerName"]["q_conv1d_1"]["ConvImplementation"] = "Pointwise"
config["LayerName"]["input_1"]["Precision"] = "fixed<32,16>"

for layer in config['LayerName'].keys():
    print('Enable tracing for layer:', layer)
    config['LayerName'][layer]['Trace'] = True

hls_model = hls4ml.converters.convert_from_keras_model(model,
                                                       hls_config=config,
                                                       output_dir='qkmodel_1/hls4ml_prj',
                                                       part='xcvu13p-flga2577-2-e')

#plot hls model
hls4ml.utils.plot_model(hls_model, show_shapes=True, show_precision=True, to_file="/home/users/russelld/L1JetTagDaniel/LayerTraces/qkmodel.png")


#Compile model, no need to convert if we are plotting performance
hls_model.compile()

# Handle Data:
with h5py.File("/home/users/russelld/L1JetTagDaniel/hls4mlModifications/10-08-23/newTestDataST30.h5", "r") as hf:
    dataset = hf["Testing Data"][:]
with h5py.File("/home/users/russelld/L1JetTagDaniel/backgroundQCD/testingDatapt30QCD.h5", "r") as hf:
    datasetQCD = hf["Testing Data"][:]
    
dataset = np.concatenate((dataset,datasetQCD)) #Stacking datasets on top of another
np.random.shuffle(dataset) #shuffling rows
   
N_PART_PER_JET = 10
N_FEAT = 14
A = dataset[:, 0 : len(dataset[0]) - 1]
b = dataset[:, len(dataset[0]) - 1]
#A = expand_dims(A, axis=3)
A = A.reshape((A.shape[0], N_PART_PER_JET, N_FEAT))
X_test = np.ascontiguousarray(A)


2024-06-24 20:55:22.500964: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-24 20:55:26.118132: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


/home/users/russelld/L1JetTagDaniel/Software/hls4ml/hls4ml/converters/__init__.py:27: UserWarning: WARNING: Pytorch converter is not enabled!
  warnings.warn("WARNING: Pytorch converter is not enabled!", stacklevel=1)


Interpreting Model
Topology:
Layer name: input_1, layer type: InputLayer, input shapes: [[None, 10, 14]], output shape: [None, 10, 14]
Layer name: q_conv1d, layer type: QConv1D, input shapes: [[None, 10, 14]], output shape: [None, 10, 10]
Layer name: q_activation, layer type: Activation, input shapes: [[None, 10, 10]], output shape: [None, 10, 10]
Layer name: q_conv1d_1, layer type: QConv1D, input shapes: [[None, 10, 10]], output shape: [None, 10, 10]
Layer name: q_activation_1, layer type: Activation, input shapes: [[None, 10, 10]], output shape: [None, 10, 10]
Layer name: global_average_pooling1d, layer type: GlobalAveragePooling1D, input shapes: [[None, 10, 10]], output shape: [None, 10]
Layer name: q_dense, layer type: QDense, input shapes: [[None, 10]], output shape: [None, 10]
Layer name: q_activation_2, layer type: Activation, input shapes: [[None, 10]], output shape: [None, 10]
Layer name: q_dense_1, layer type: QDense, input shapes: [[None, 10]], output shape: [None, 1]
Layer 

Done


In [3]:
#!pip install seaborn

In [4]:
#begin profiling
import hls4ml.model.profiling

In [5]:
y_hls, hls4ml_trace = hls_model.trace(X_test)
keras_trace = hls4ml.model.profiling.get_ymodel_keras(model, A)

Recompiling myproject with tracing
Writing HLS project


Done
3409/3409 [==============================] - 3s 867us/step
Done taking outputs for Keras model.


In [8]:
for LAYER in hls4ml_trace.keys():
    plt.figure()
    plt.scatter(hls4ml_trace[LAYER].flatten(), keras_trace[LAYER].flatten())
    min_x = min(np.amin(hls4ml_trace[LAYER]), np.amin(keras_trace[LAYER]))
    max_x = max(np.amax(hls4ml_trace[LAYER]), np.amax(keras_trace[LAYER]))
    plt.plot([min_x, max_x], [min_x, max_x], c='gray')
    plt.xlabel('hls4ml {}'.format(LAYER))
    #plt.xlabel('hls4ml {}'.format(LAYER))
    plt.ylabel('QKeras {}'.format(LAYER))
    plt.savefig(f"/home/users/russelld/L1JetTagDaniel/LayerTraces/profiling_{LAYER}.png")   